In [23]:
import sqlite3
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine , inspect


In [24]:
connection = sqlite3.connect("covid.sqlite")

In [25]:
query = "SELECT * from daily_cases_by_state"
df = pd.read_sql_query(query,connection)
df

,date,AK,AL,AR,AS,AZ,CA,CO,CT,DC,...,TN,TX,UT,VA,VI,VT,WA,WI,WV,WY
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,416,177,2733,404,0,1284,3352,1055,494,51,...,1598,7822,729,1549,49,115,733,780,232,89
416,417,140,922,403,0,1154,3504,1351,878,196,...,1514,8028,0,1300,9,199,795,855,261,69
417,418,141,811,570,0,2276,4659,1633,830,108,...,1445,6853,1160,1652,10,133,854,721,287,79
418,419,0,524,327,0,1735,4452,1108,0,151,...,1312,5570,570,1477,0,145,778,614,326,0


In [26]:
# create an engine to connect to your SQLite database
engine = create_engine("sqlite:///covid.sqlite")

# create an inspector object to inspect the database
inspector = inspect(engine)

# get a list of the tables in the database
table_names = inspector.get_table_names()

# print the table names
print(table_names)

['cumulative_hospitalized_by_state', 'daily_cases_by_state', 'daily_cases_update', 'daily_cases_us', 'daily_cumulative_discharged_cases_by_state', 'daily_current_hospitalized_cases_by_state', 'daily_deaths_by_state', 'daily_deaths_by_state.csv', 'daily_hospitalizations_by_state', 'daily_hospitalizations_by_state_cumulative', 'daily_recoveries_by_state_cumulative', 'in_icu_currently_by_state', 'national_history']


In [27]:
# This code aggregates data from each of the tables into state specific tables.

import sqlite3
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Connect to the SQLite database
engine = create_engine("sqlite:///covid.sqlite")
Session = sessionmaker(bind=engine)
session = Session()

# Define SQL queries to read the tables
query_cases = "SELECT * FROM daily_cases_by_state"
query_hospitalizations = "SELECT * FROM daily_hospitalizations_by_state"
query_deaths = "SELECT * FROM daily_deaths_by_state"
query_cumulative_hospitalizations = "SELECT * FROM cumulative_hospitalized_by_state"
query_cumulative_discharged_cases = "SELECT * FROM daily_cumulative_discharged_cases_by_state"
query_current_hospitalized_cases = "SELECT * FROM daily_current_hospitalized_cases_by_state"
query_daily_hospitalizations_cumulative = "SELECT * FROM daily_hospitalizations_by_state_cumulative"
query_daily_recoveries_cumulative = "SELECT * FROM daily_recoveries_by_state_cumulative"


# Read the tables from the database into pandas DataFrames
daily_cases = pd.read_sql_query(query_cases, con=engine)
daily_hospitalizations = pd.read_sql_query(query_hospitalizations, con=engine)
daily_deaths = pd.read_sql_query(query_deaths, con=engine)
cumulative_hospitalizations = pd.read_sql_query(query_cumulative_hospitalizations, con=engine)
cumulative_discharged_cases = pd.read_sql_query(query_cumulative_discharged_cases, con=engine)
current_hospitalized_cases = pd.read_sql_query(query_current_hospitalized_cases, con=engine)
daily_hospitalizations_cumulative = pd.read_sql_query(query_daily_hospitalizations_cumulative, con=engine)
daily_recoveries_cumulative = pd.read_sql_query(query_daily_recoveries_cumulative, con=engine)

# Get the list of state abbreviations from the column names
state_abbreviations = [col for col in daily_cases.columns if col not in ["date"]]

# Define the state-specific table schema
Base = declarative_base()

def create_state_specific_table(state):
    class StateSpecific(Base):
        __tablename__ = f"{state} Aggregated"
        date = Column(Integer, primary_key=True)
        cases = Column(Float)
        hospitalizations = Column(Float)
        deaths = Column(Float)
        cumulative_hospitalizations = Column(Float)
        cumulative_discharged_cases = Column(Float)
        current_hospitalized_cases = Column(Float)
        daily_hospitalizations_cumulative = Column(Float)
        daily_recoveries_cumulative = Column(Float)
    return StateSpecific

for state in state_abbreviations:
    # Merge data for the current state
    merged_data = pd.DataFrame()
    merged_data["date"] = daily_cases["date"]
    merged_data["cases"] = daily_cases[state]
    merged_data["hospitalizations"] = daily_hospitalizations[state]
    merged_data["deaths"] = daily_deaths[state]
    merged_data["cumulative_hospitalizations"] = cumulative_hospitalizations[state]
    merged_data["cumulative_discharged_cases"] = cumulative_discharged_cases[state]
    merged_data["current_hospitalized_cases"] = current_hospitalized_cases[state]
    merged_data["daily_hospitalizations_cumulative"] = daily_hospitalizations_cumulative[state]
    merged_data["daily_recoveries_cumulative"] = daily_recoveries_cumulative[state]
    
    # Create a new table for the current state
    StateSpecific = create_state_specific_table(state)
    Base.metadata.create_all(engine)

    # Write the merged data to the new table
    merged_data.to_sql(name=f"{state} Aggregated", con=engine, if_exists="replace", index=False)

# Close the connection
session.close()



/Users/cameronleslie/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:41: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.StateSpecific, and will be replaced in the string-lookup table.


In [28]:
#Checking new tables are included


# create an engine to connect to your SQLite database
engine = create_engine("sqlite:///covid.sqlite")

# create an inspector object to inspect the database
inspector = inspect(engine)

# get a list of the tables in the database
table_names = inspector.get_table_names()

# print the table names
print(table_names)

['AK Aggregated', 'AL Aggregated', 'AR Aggregated', 'AS Aggregated', 'AZ Aggregated', 'CA Aggregated', 'CO Aggregated', 'CT Aggregated', 'DC Aggregated', 'DE Aggregated', 'FL Aggregated', 'GA Aggregated', 'GU Aggregated', 'HI Aggregated', 'IA Aggregated', 'ID Aggregated', 'IL Aggregated', 'IN Aggregated', 'KS Aggregated', 'KY Aggregated', 'LA Aggregated', 'MA Aggregated', 'MD Aggregated', 'ME Aggregated', 'MI Aggregated', 'MN Aggregated', 'MO Aggregated', 'MP Aggregated', 'MS Aggregated', 'MT Aggregated', 'NC Aggregated', 'ND Aggregated', 'NE Aggregated', 'NH Aggregated', 'NJ Aggregated', 'NM Aggregated', 'NV Aggregated', 'NY Aggregated', 'OH Aggregated', 'OK Aggregated', 'OR Aggregated', 'PA Aggregated', 'PR Aggregated', 'RI Aggregated', 'SC Aggregated', 'SD Aggregated', 'TN Aggregated', 'TX Aggregated', 'UT Aggregated', 'VA Aggregated', 'VI Aggregated', 'VT Aggregated', 'WA Aggregated', 'WI Aggregated', 'WV Aggregated', 'WY Aggregated', 'cumulative_hospitalized_by_state', 'daily_case

In [34]:
#This code exports all the newly created state aggregate data into a new folder.

import os
import pandas as pd
from sqlalchemy import create_engine

# Connect to the SQLite database
engine = create_engine("sqlite:///covid.sqlite")

# List of state abbreviations
state_abbreviations = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

# Create a new folder called "State Aggregates" if it doesn't already exist
os.makedirs("State Aggregates", exist_ok=True)

# Iterate through each state abbreviation
for state in state_abbreviations:
    # Define SQL query to read the aggregated table for the current state
    query_state_aggregated = f"SELECT * FROM '{state} Aggregated'"
    
    # Read the aggregated table from the database into a pandas DataFrame
    state_aggregated_data = pd.read_sql_query(query_state_aggregated, con=engine)
    
    # Export the contents of the aggregated table to a CSV file in the "State Aggregates" folder
    state_aggregated_data.to_csv(os.path.join("State Aggregates", f"{state}_Aggregated.csv"), index=False)

# Close the connection
engine.dispose()
